In [27]:
import pandas as pd 
import numpy as np 
import torch
from torch import nn
from datetime import datetime
import itertools

mode = "train" | "predict"

In [20]:
learning_rate = 1e-6
num_iterations = 2000
mode = "test"
save_model = False
load_model = False
model_save_path = "./model" #it will have automatically appended the current date
model_load_path = "model2024-01-25_16_38.pt"

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
device

'cuda'

In [5]:
class Classifier(nn.Module):
    def __init__(self, dim_input, num_classes):
        super().__init__()
        #self.classifier = nn.Linear(dim_input, num_classes)
        self.model = nn.Sequential(
            nn.Linear(dim_input, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
            nn.Softmax(dim=1)
        )
#should return logits and features
#features is ignored for now
    def forward(self, x):
        return self.model(x)
       # return self.classifier(x), {}


In [6]:
df = pd.read_pickle("./comet_samsum_test_z_entire.pkl")

In [7]:
ohe_dict= \
{'xNeed': np.array([1,0,0,0,0], dtype=np.float32),
 'xIntent': np.array([0,1,0,0,0], dtype=np.float32),
 'HinderedBy': np.array([0,0,1,0,0], dtype=np.float32),
 'xWant':  np.array([0,0,0,1,0], dtype=np.float32),
 'xReason':  np.array([0,0,0,0,1], dtype=np.float32)}

In [8]:
df["cs_type_ohe"] = df["cs_type"].map(ohe_dict)
preprocessed = df[["sample_id","sentence_id", "cs_type_ohe","cs_encoded", "cos_similary_cs_summmary"]]


In [9]:
set(preprocessed["cs_encoded"].transform(len))

{384}

In [11]:
preprocessed["complete_target_X"] =preprocessed[["cs_type_ohe","cs_encoded"]].apply(lambda row: np.append(row[0],row[1]), axis=1)

C:\Users\NonAv\AppData\Local\Temp\ipykernel_2628\3403830363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed["complete_target_X"] =preprocessed[["cs_type_ohe","cs_encoded"]].apply(lambda row: np.append(row[0],row[1]), axis=1)


In [12]:
new_df = preprocessed.groupby(['sample_id','sentence_id']).agg(X=("complete_target_X",lambda x: list(itertools.chain.from_iterable(x))),Y=('cos_similary_cs_summmary',list))

new_df.reset_index() 

,sample_id,sentence_id,X,Y
0,13611370,0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.005920287, -0.0092...","[0.3164701759815216, 0.3950572609901428, 0.384..."
1,13611370,1,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.041891553, 0.07421...","[0.40869206190109253, 0.41971734166145325, 0.4..."
2,13611370,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.06959674, 0.007380...","[0.31288936734199524, 0.35756880044937134, 0.3..."
3,13611370,3,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.020754024, -0.0092...","[0.07747259736061096, 0.08857818692922592, 0.0..."


In [21]:
model = Classifier(dim_input=9725, num_classes=25).to(device)
if load_model:
    model.load_state_dict(torch.load(model_load_path))

In [22]:
sm =  nn.Softmax(dim=1)

In [23]:
X = torch.tensor(new_df["X"]).to(device)
Y = sm(torch.tensor(new_df["Y"])).to(device)

In [24]:
if mode == "train":
    for t in range(num_iterations):
        # Forward pass: compute predicted y using operations on Tensors.
        y_pred = model(X)

        # Compute and print loss using operations on Tensors.
        # Now loss is a Tensor of shape (1,)
        # loss.item() gets the scalar value held in the loss.
        loss = (y_pred - Y).pow(2).sum()
        if t % 100 == 99:
            print(t, loss.item())

        # Use autograd to compute the backward pass. This call will compute the
        # gradient of loss with respect to all Tensors with requires_grad=True.
        # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
        # the gradient of the loss with respect to a, b, c, d respectively.
        loss.backward()

99 0.0023943614214658737
199 0.0023943614214658737
299 0.0023943614214658737
399 0.0023943614214658737
499 0.0023943614214658737
599 0.0023943614214658737
699 0.0023943614214658737
799 0.0023943614214658737
899 0.0023943614214658737
999 0.0023943614214658737
1099 0.0023943614214658737
1199 0.0023943614214658737
1299 0.0023943614214658737
1399 0.0023943614214658737
1499 0.0023943614214658737
1599 0.0023943614214658737
1699 0.0023943614214658737
1799 0.0023943614214658737
1899 0.0023943614214658737
1999 0.0023943614214658737


In [25]:
if save_model:
    torch.save(model.state_dict(), f"{model_save_path}{datetime.today().strftime('%Y-%m-%d_%H_%M')}.pt")

In [62]:
def test(X,Y, model, loss_fn):
    test_loss, correct = 0, 0
    with torch.no_grad():
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        test_loss = loss_fn(pred, Y)
        #correct = 
        for i,similarities in enumerate(pred):
            correct +=int(similarities.argmax() == Y[i].argmax())
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [30]:
mode = "test"

In [64]:
if mode == "test" or mode == "train":
    test(X,Y, model, lambda y_pred, Y: (y_pred - Y).pow(2).sum())

Test Error: 
 Accuracy: 0.0%, Avg loss: 0.002394 

